In [ ]:
!pip install -e .. > /dev/null 2>&1

In [ ]:
import os
import json
import gdown
import shutil

from astrohack.gdown_utils import gdown_data
from astrohack.gdown_utils import build_folder_structure

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 20]

In [ ]:
# Build folder structure & download 
import astrohack

base_name = 'ea25_cal_small_'

datafolder = 'data'
resultsfolder = 'results'

build_folder_structure(datafolder, resultsfolder)


astrohack.gdown_utils.download('ea25_cal_small_before_fixed.split.ms', folder=datafolder, unpack=True)
astrohack.gdown_utils.download('ea25_cal_small_after_fixed.split.ms', folder=datafolder, unpack=True)

astrohack.gdown_utils.download(file='extract_holog_verification.json')
astrohack.gdown_utils.download(file='holog_numerical_verification.json')


In [ ]:
from astrohack.astrohack_client import astrohack_local_client

log_parms = {'log_level':'DEBUG'}

client = astrohack_local_client(cores=2, memory_limit='8GB', log_parms=log_parms)

In [ ]:
def test_holog_obs_dictionary(holog_obs_dict):
    import os
    import json
    import copy
    
    import numpy as np

    from astrohack._utils._tools import _jsonify

    ref_holog_obj = {}
    ref_holog_obj = copy.deepcopy(holog_obs_dict)

    _jsonify(ref_holog_obj)

    with open(".holog_obs_dict.json") as json_file:
        holog_obj = json.load(json_file)
                          
    assert holog_obj == ref_holog_obj, "Error: holog_obs_descrition dictionary has changes unexpectedly."
    
def test_holog_diagnostics(json_data, tolerance=1e-7):
    import json
    
    with open("holog_numerical_verification.json") as file:
        reference_dict = json.load(file)
        
    cell_size = reference_dict["vla"]['cell_size'][1]
    grid_size = float(reference_dict["vla"]['grid_size'][1])
    
    json_data['cell_size'] = np.abs(float(json_data['cell_size']))
    
    cell_size = np.abs(float(cell_size))
    
    assert relative_difference(json_data['cell_size'], cell_size) < tolerance, "Unexpected change in cell_size occured."
    assert relative_difference(np.sqrt(int(json_data['n_pix'])), grid_size) < tolerance, "Unexpected change in grid_size occured."

    
def test_center_pixel(file, antenna, ddi, reference_center_pixels, number_of_digits=7):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = np.squeeze(mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2])
    beam_center_pixels = np.squeeze(mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2])
    
    aperture_ref = list(map(complex, reference_center_pixels['aperture']))
    beam_ref = list(map(complex, reference_center_pixels['beam']))
    
    for i in range(len(aperture_ref)):        
        assert relative_difference(
            aperture_ref[i].real, 
            aperture_center_pixels[i].real
        ) < 1e-6, "There has been a shift in aperture center pixel value(s)"
        
        assert relative_difference(
            beam_ref[i].real, 
            beam_center_pixels[i].real
        ) < 1e-6, "There has been a shift in beam center pixel value(s)"
                
        assert relative_difference(
            aperture_ref[i].imag, 
            aperture_center_pixels[i].imag
        ) < 1e-6, "There has been a shift in aperture center pixel value(s)"
        assert relative_difference(
            beam_ref[i].imag, 
            beam_center_pixels[i].imag
        ) < 1e-6, "There has been a shift in beam center pixel value(s)"
        
              
def get_center_pixel(file, antenna, ddi):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2]
    beam_center_pixels = mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2]
    
    return np.squeeze(aperture_center_pixels), np.squeeze(beam_center_pixels)

def compare_float_values(result, reference, decimals):
    result_list = list(map(str, str(result)))
    reference_list = list(map(str, str(reference)))
    
    return result_list[:decimals] == reference_list[:decimals]

def relative_difference(result, expected):
    return 2*np.abs(result - expected)/(abs(result) + abs(expected))

In [ ]:
from astrohack.extract_pointing import extract_pointing

before_ms = datafolder+'/'+ base_name + 'before_fixed.split.ms'
after_ms =  datafolder+'/'+ base_name + 'after_fixed.split.ms'

before_point = resultsfolder+'/before.split.point.zarr'
after_point = resultsfolder+'/after.split.point.zarr'

pnt_mds = extract_pointing(
      ms_name=before_ms,
      point_name=before_point,
      parallel=False,
      overwrite=True
)

extract_pointing(
      ms_name=after_ms,
      point_name=after_point,
      parallel=False,
      overwrite=True
)

In [ ]:
from astrohack.extract_holog import extract_holog
import numpy as np

before_holog = resultsfolder+'/before.split.holog.zarr'
after_holog = resultsfolder+'/after.split.holog.zarr'

with open("extract_holog_verification.json") as file:
    json_dict = json.load(file)
    
holog_obs_dict = json_dict["vla"]

extract_holog(
    ms_name=before_ms, 
    point_name=before_point,
    holog_name=before_holog,
    ddi=[0],
    data_column='CORRECTED_DATA',
    parallel=False,
    overwrite=True
)

test_holog_obs_dictionary(holog_obs_dict["before"])

with open(resultsfolder+'/before.split.holog.zarr/.holog_attr') as attr_file:
    holog_attr = json.load(attr_file)
    
test_holog_diagnostics(
    json_data=holog_attr,
    tolerance=2e-5
)


extract_holog(
    ms_name=after_ms,
    point_name=after_point,
    holog_name=after_holog,
    data_column='CORRECTED_DATA',
    parallel=False,
    overwrite=True
)

test_holog_obs_dictionary(holog_obs_dict["after"])


with open(resultsfolder+'/after.split.holog.zarr/.holog_attr') as attr_file:
    holog_attr = json.load(attr_file)
    
test_holog_diagnostics(
    json_data=holog_attr,
    tolerance=2e-5
)


In [ ]:
import numpy as np
from astrohack import holog

with open("holog_numerical_verification.json") as file:
    reference_dict = json.load(file)

grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
scan_average = True

holog(
    holog_name=before_holog, 
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)

test_center_pixel(
    file="results/before.split.image.zarr", 
    antenna="ant_ea25", 
    ddi="ddi_0", 
    reference_center_pixels=reference_dict["vla"]["pixels"]["before"]
)

holog(
    holog_name=after_holog, 
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    scan_average = scan_average,
    overwrite=True,
    phase_fit=True,
    apply_mask=True,
    to_stokes=True,
    parallel=True
)

test_center_pixel(
    file="results/after.split.image.zarr", 
    antenna="ant_ea25", 
    ddi="ddi_0", 
    reference_center_pixels=reference_dict["vla"]["pixels"]["after"]
)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import scipy
import matplotlib.patches as patches

from scipy import constants

from astrohack.dio import open_image

before_image = resultsfolder+'/before.split.image.zarr'
after_image = resultsfolder+'/after.split.image.zarr'

plt.close('all')

chan = 0

ds = open_image("results/before.split.image.zarr")['ant_ea25']['ddi_0']

wavelength = scipy.constants.speed_of_light/ds.chan.values[chan]
l = ds.l.values
m = ds.m.values
u = ds.u.values*wavelength
v = ds.v.values*wavelength

print(u.min(),u.max())

plt.figure()
plt.imshow(np.abs(ds.BEAM[0,chan,0,:,:]),extent=[l.min(), l.max(), m.min(), m.max()])
plt.colorbar()
plt.show()

circle = patches.Circle((0,0), 12.5, fill=False, color='white', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()
plt.imshow(np.abs(ds.APERTURE[0,chan,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])

plt.show()

ds

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

before_panel = panel(
    image_name=before_image, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)
after_panel = panel(
    image_name=after_image, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)

In [ ]:
import numpy as np
from astrohack.dio import open_panel

def relative_difference(mean, expected):  
    return 2*np.abs(mean - expected)/(abs(mean) + abs(expected))

def verify_panel_shifts(
    panel_list=['3-4', '5-27', '5-37', '5-38'], 
    expected_shift=np.array([-100, 75, 0, 150]),
    ref_mean_shift = np.array([-112.23760235, 73.09423151, -1.52957784, 138.96735818]),
    antenna='ant_ea25',
    ddi='ddi_0'
):
    
    M_TO_MILS = 39370.1
    
    before_mds = open_panel('results/before.split.panel.zarr')
    after_mds = open_panel('results/after.split.panel.zarr')
    
    before_shift = before_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    after_shift = after_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    
    difference = after_shift - before_shift
    
    mean_shift = np.mean(difference, axis=1)
    
    delta_mean_shift = np.abs(mean_shift - expected_shift)
    delta_ref_shift = np.abs(ref_mean_shift - expected_shift)
        
    delta_shift = delta_mean_shift - delta_ref_shift  # New corrections - old corrections --> delta if delta < 0 ==> we improved.
    relative_shift = relative_difference(delta_mean_shift, delta_ref_shift)
       
    if np.any(relative_shift > 1e-6): 
        print("There were changes!")
        for i, delta in enumerate(delta_shift):
            if delta < 0:
                print("{panel}, improved by {delta} mils".format(panel=panel_list[i], delta=delta))
            else:
                print("{panel}, got worse by {delta} mils".format(panel=panel_list[i], delta=delta))

In [ ]:
verify_panel_shifts()